## Pulling ASO from SnowEx database

This code was built from the SnowEx hackweek database tutorial

### Connect to database:

In [1]:
# Import the function to get connect to the db
from snowexsql.db import get_db

# Import our class for the points table
from snowexsql.data import ImageData

# import this to use define sql functions (e.g. postgis!)
from sqlalchemy.sql import func 

# Import this to convert to a rasterio object for easy plotting
from snowexsql.conversions import raster_to_rasterio 

# Import a convenient function to plot with 
from rasterio.plot import show

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

In [2]:
# Import the function to investigate a table
from snowexsql.db import get_table_attributes

# Use the function to see what columns are available to use. 
db_columns = get_table_attributes(ImageData)

# Print out the results nicely
print("These are the available columns in the table:\n \n* {}\n".format('\n* '.join(db_columns)))

These are the available columns in the table:
 
* date
* date_accessed
* description
* doi
* instrument
* metadata
* observers
* raster
* registry
* site_name
* time_created
* time_updated
* type
* units



### What to filter?

In [3]:
# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

# Get the unique datanames in the table
results = session.query(ImageData.type).distinct().all()
print('Available types = {}'.format(', '.join([r[0] for r in results])))

# Get the unique instrument in the table
results = session.query(ImageData.instrument).distinct().all()
print('\nAvailable Instruments = {}'.format(', '.join([str(r[0]) for r in results])))

# Get the unique dates in the table
results = session.query(ImageData.date).distinct().all()
print('\nAvailable Dates = {}'.format(', '.join([str(r[0]) for r in results])))

# Get the unique sites in the table
results = session.query(ImageData.site_name).distinct().all()
print('\nAvailable sites = {}'.format(', '.join([str(r[0]) for r in results])))

session.close()

Available types = swe, DEM, insar amplitude, insar correlation, depth, insar interferogram real, insar interferogram imaginary

Available Instruments = UAVSAR, L-band InSAR, lidar

Available Dates = 2020-02-13, None, 2020-02-21, 2020-02-12, 2020-01-31, 2020-02-01, 2020-02-02, 2020-03-11

Available sites = None, idaho, Grand Mesa


### Query

In [ ]:
# import the handy function to convert raster db results to rasterio
from snowexsql.conversions import raster_to_rasterio

# Import the SQL function to access PostGIS functions
from sqlalchemy.sql import func

# Import rasterio for Writing
import rasterio 

# Grab a session
engine, session = get_db(db_name)

# Query 1 raster tile and convert it to a geotiff
result = session.query(func.ST_AsTiff(ImageData.raster)).limit(1).all()

# Convert the dataset to a rasterio dataset
dataset = raster_to_rasterio(session, result)

# Copy the profile/tiff metadata (not to be confused with the database metadata)
profile = dataset[0].profile

# Write to a file 
with rasterio.open('raster.tif', 'w', **profile) as dst:
    dst.write(dataset[0].read(1), 1)

session.close()